In [3]:
import torch
from diffusers import StableDiffusionPipeline


model_id = "OFA-Sys/small-stable-diffusion-v0/"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "an apple, 4k"
image = pipe(prompt).images[0]  
    
image.save("apple.png")

/workspaces/pokemon-ai-ze/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
safety_checker/model.safetensors not found
Loading pipeline components...:  43%|████▎     | 3/7 [00:02<00:03,  1.07it/s]

: 

In [2]:
pip install --upgrade diffusers transformers scipy accelerate


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
